In [25]:
from postgres_connect import *
import requests

In [22]:
query = '''
select r.rider_id, node_node_id
from rider_pip 
left join application_db.rider as r on r.locus_rider_id = rider_pip.rider_id
where 
        pip_status = 'closed' 
and     pip_result = 'failed'
order by node_node_id
'''

In [23]:
path = '/Users/apple/Documents/tunnel-ssh.cer'
# printthis('yes')
conn = get_conn('Yes',path)
df_block = pd.read_sql(query,conn)
conn.close()

Tunnel Started
Connection Made


In [36]:
url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9001/rider/delist'
headers_base = {
    'name': 'void',  # Change this value if needed
    'Content-Type': 'application/json',
}

for index, row in df_block.iterrows():
    headers = headers_base.copy()
    headers['node_id'] = str(row['node_node_id'])

    data = {
        'delistType': 'BLACKLIST',
        'riderIds': [str(row['rider_id'])],
    }

    response = requests.put(url, headers=headers, json=data)

    try:
        json_response = response.json()
        message = json_response.get('message', '')
        print(f"Request for rider_id {row['rider_id']} - {message}")

        data = json_response.get('data', [])
        for item in data:
            rider_id = item.get('riderId', '')
            rider_name = item.get('riderName', '')
            error_message = item.get('message', '')
            print(f"    Rider ID: {rider_id}, Rider Name: {rider_name}, Error Message: {error_message}")

    except ValueError:
        print("Invalid JSON response")
    
print("Rider blocking completed updating database")


Request for rider_id 3182 - Success
Request for rider_id 1780 - Partial Success
    Rider ID: 1780, Rider Name: Pavan Kumar R, Error Message: Some error occurred
Request for rider_id 3281 - Success
Request for rider_id 2130 - Success
Request for rider_id 3193 - Success
Request for rider_id 3509 - Success
Request for rider_id 3195 - Success
Request for rider_id 2825 - Success
Request for rider_id 1339 - Partial Success
    Rider ID: 1339, Rider Name: Sunil Gadge, Error Message: Some error occurred
Request for rider_id 3137 - Success
Request for rider_id 3508 - Success
Request for rider_id 3507 - Success
Request for rider_id 3480 - Success
Request for rider_id 3292 - Success
Request for rider_id 1114 - Partial Success
    Rider ID: 1114, Rider Name: Himanshu Pal, Error Message: Some error occurred
Request for rider_id 2203 - Partial Success
    Rider ID: 2203, Rider Name: Ankit Kumar , Error Message: Some error occurred
Request for rider_id 1177 - Success
Request for rider_id 3353 - Succ

In [32]:
conn = get_conn('Yes',path)
cursor = conn.cursor()
update_query1 = '''
    UPDATE rider_pip
    SET pip_status = 'final'
    WHERE pip_status = 'closed';
'''
update_query2 = '''
delete from rider_pip where pip_status = 'running' and rider_id in 
(select rider_id from rider_pip where pip_status = 'final' and pip_result = 'failed')
'''
cursor.execute(update_query1)
print("Updated all closed status to final")
cursor.execute(update_query2)
print("Deleted blocked riders from running pip")
conn.commit()
cursor.close()
conn.close()
print("Updated all closed status to final")

Tunnel Started
Connection Made
Updated all closed status to final
Deleted blocked riders from running pip
Updated all closed status to final
